In [1]:
import numpy as np
import cocoex
from skopt import gp_minimize 
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 创建测试套件并获取问题实例
f = []
suite = cocoex.Suite("bbob", "year:2023", "dimensions: 5")
bounds = [(-5., 5.), ] * 5
for i in range(24):
    f.append(suite.get_problem(i*15))

In [3]:
n_repeats = 10
n_calls = 200
n_init = 20

In [4]:
for i in range(8,11,1):
    results = []
    premin = []
    with tqdm(total=n_repeats*n_calls,desc = "Optimizing") as pbar:
        def callback(result):
            if len(result.models)==0:
                premin.append(result.x_iters[-1])
            else:
                res = gp_minimize(f[i],bounds,n_initial_points=0,acq_func="LCB",n_calls=1,random_state=1,kappa=0,x0=result.x_iters,y0=result.func_vals)
                premin.append(res.x_iters[-1])
            pbar.update(1)
        results = [gp_minimize(f[i],bounds,n_initial_points=n_init,initial_point_generator='lhs',acq_func="EI",n_calls=n_calls,random_state=n,callback=callback) for n in range(n_repeats)]   
    for result in results:
        with open("f"+str(i+1)+".csv","a") as file:
            np.savetxt(file,result.x_iters,delimiter=',')

    with open("f"+str(i+1)+"_pre.csv","a") as file:
        np.savetxt(file,premin,delimiter=',') 

    with open("fmin.csv","a") as file:
        file.write(str(f[i].best_observed_fvalue1))
        file.write('\n')
    f[i].free

Optimizing: 100%|██████████| 2000/2000 [1:20:41<00:00,  2.42s/it]


: 